# Machine Learning Cambio Colombia

# 1. Cálculos de Predicciones
Este bloque de código realiza los cálculos necesarios para generar las variables que se usarán en el modelo predictivo:
- Recencia de la última compra.
- Frecuencia de pagos.
- Valor total de suscripción.
- Número de renovaciones.


In [24]:
import pandas as pd
from datetime import datetime

# Cargar el archivo Excel
df = pd.read_excel('/Users/juandiegoruizgonzalez/Desktop/Machine Learning Cambio colombia/ML_Cambio colombia.xlsx')

# Verificar las primeras filas para revisar las columnas
print(df.head())

# Convertir 'Fecha compra' a tipo datetime, forzando errores a NaT si no se puede convertir
df['Fecha compra'] = pd.to_datetime(df['Fecha compra'], errors='coerce')

# Eliminar filas con valores nulos en las columnas importantes
df.dropna(subset=['Fecha compra', 'Correo', 'Valor'], inplace=True)

# Calcular recencia de la última compra (días desde la última compra)
today = datetime.today()
df['Recencia'] = (today - df['Fecha compra']).dt.days

# Calcular el valor total de la suscripción por cliente (sumar 'Valor' por 'Correo')
total_value = df.groupby('Correo')['Valor'].sum().reset_index()
total_value.rename(columns={'Valor': 'Valor total'}, inplace=True)

# Calcular el número de renovaciones (número de transacciones por 'Correo')
renewals = df.groupby('Correo').size().reset_index(name='Número de renovaciones')

# Unir las variables calculadas con el dataframe original
df = pd.merge(df, total_value, on='Correo', how='left')
df = pd.merge(df, renewals, on='Correo', how='left')

# Mostrar los primeros registros para verificar
df.head()


           Nombre         Apellido Tipo doc Documento  \
0        Fernando  Otoya Dominguez   CÈdula  16585012   
1   Rosa Victoria  Campo Rodriguez   CÈdula  35459777   
2          Wilma      Zafra Turbay   CÈdula  21069048   
3  Carlos Alfonso  Negret Mosquera   CÈdula  10541955   
4          Hernan           Onatra   CÈdula  12236147   

                       Correo     Celular    Estado No factura Fecha compra  \
0               fod@siesa.com  3154913615  Inactiva  FV-2 - 19   2021-11-05   
1        toyacampos@gmail.com  3002677060  Inactiva  FV-2 - 23   2021-11-05   
2       wilmazafrat@gmail.com  3102100831  Inactiva  FV-2 - 24   2021-11-05   
3  negretmosquera@hotmail.com  3112599185  Inactiva  FV-2 - 25   2021-11-05   
4      hernanonatra@gmail.com  3203067981  Inactiva  FV-2 - 26   2021-11-05   

      Valor  ... Saldo No de pago      Pais     Departamento  \
0  30000000  ...     0          0  Colombia  Valle del Cauca   
1   1000000  ...     0          0       NaN           

,Nombre,Apellido,Tipo doc,Documento,Correo,Celular,Estado,No factura,Fecha compra,Valor,...,Departamento,Ciudad,Direccion,id_cliente,Primera transacciÛn,⁄ltimo vencimiento,id_suscripcion_transaccion,Recencia,Valor total,Número de renovaciones
0,Fernando,Otoya Dominguez,CÈdula,16585012,fod@siesa.com,3154913615,Inactiva,FV-2 - 19,2021-11-05,30000000,...,Valle del Cauca,Cr 3 Oeste 9-83 Ap 402 A Ed Arboleda Reservado,Cali,7975,Si,Si,2,1343,30000000,1
1,Rosa Victoria,Campo Rodriguez,CÈdula,35459777,toyacampos@gmail.com,3002677060,Inactiva,FV-2 - 23,2021-11-05,1000000,...,NaN,NaN,NaN,22231,Si,Si,3,1343,1000000,1
2,Wilma,Zafra Turbay,CÈdula,21069048,wilmazafrat@gmail.com,3102100831,Inactiva,FV-2 - 24,2021-11-05,1000000,...,"Bogot·, D.C.",Carrera 10 No. 82-66,"Bogot·, D.C.",22899,Si,No,4,1343,1069000,2
3,Carlos Alfonso,Negret Mosquera,CÈdula,10541955,negretmosquera@hotmail.com,3112599185,Inactiva,FV-2 - 25,2021-11-05,1000000,...,"Bogot·, D.C.",Cra 7 # 81-25,"Bogot·, D.C.",18152,Si,No,5,1343,1152000,3
4,Hernan,Onatra,CÈdula,12236147,hernanonatra@gmail.com,3203067981,Inactiva,FV-2 - 26,2021-11-05,1000000,...,"Bogot·, D.C.",Cra 63 #22-A-41 Interior 1 Apto 402,"Bogot·, D.C.",9843,Si,Si,6,1343,1000000,1


# 2. Entrenamiento del Modelo
En este bloque de código:
- Preparamos los datos.
- Creamos un modelo predictivo utilizando regresión logística.
- Entrenamos el modelo con las variables generadas.


In [25]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

# Seleccionar las variables clave para el modelo
df_model = df[['Recencia', 'Valor total', 'Número de renovaciones', 'Plan de pago']]

# Convertir 'Plan de pago' a variable numérica (1: Recurrente, 0: Corriente)
df_model['Plan de pago'] = df_model['Plan de pago'].apply(lambda x: 1 if x == 'Recurrente' else 0)

# Dividir los datos en características (X) y variable objetivo (y)
X = df_model[['Recencia', 'Valor total', 'Número de renovaciones', 'Plan de pago']]
y = df_model['Plan de pago']  # Suponemos que estamos prediciendo la probabilidad de que un cliente sea recurrente

# Escalar las características
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Dividir en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Crear y entrenar el modelo
model = LogisticRegression()
model.fit(X_train, y_train)

# Evaluar el modelo
y_pred = model.predict(X_test)

# Mostrar los resultados
print("Reporte de clasificación:")
print(classification_report(y_test, y_pred))
print("Matriz de confusión:")
print(confusion_matrix(y_test, y_pred))


Reporte de clasificación:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      8390
           1       1.00      1.00      1.00     10170

    accuracy                           1.00     18560
   macro avg       1.00      1.00      1.00     18560
weighted avg       1.00      1.00      1.00     18560

Matriz de confusión:
[[ 8390     0]
 [    0 10170]]


/var/folders/0p/vctscwcd3sjgx6r1h4npy00w0000gn/T/ipykernel_6967/1028177028.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_model['Plan de pago'] = df_model['Plan de pago'].apply(lambda x: 1 if x == 'Recurrente' else 0)


# 3. Modelo de Predicción Personalizado
Este bloque de código implementa un modelo de predicción basado en una fórmula personalizada que calcula la probabilidad de renovación de un cliente:
- Utiliza las siguientes variables:
  - **Número de renovaciones**
  - **Valor total**
  - **Plan de pago**
  - **Recencia**
- Los pesos asignados a cada variable son:
  - **Número de renovaciones**: 0.4
  - **Valor total**: 0.3
  - **Plan de pago**: 0.2
  - **Recencia**: 0.1
- La probabilidad de renovación se normaliza para estar entre 0 y 1.



In [26]:
# Asegurarse de que 'Plan de pago' esté convertido a valores numéricos (si no se ha hecho)
df_new = df[['Recencia', 'Valor total', 'Número de renovaciones', 'Plan de pago']].copy()
df_new['Plan de pago'] = df_new['Plan de pago'].apply(lambda x: 1 if x == 'Recurrente' else 0)

# Seleccionar las columnas que deben ser predichas (incluir 'Plan de pago' para la predicción)
X_new = df_new[['Recencia', 'Valor total', 'Número de renovaciones', 'Plan de pago']]

# Calcular las probabilidades de renovación
df['Probabilidad renovación'] = df.apply(calcular_probabilidad, axis=1)

# Definir un umbral para la predicción (por ejemplo, 0.5)
df['Predicción renovación'] = df['Probabilidad renovación'].apply(lambda x: 1 if x > 0.5 else 0)

# Verificar las primeras predicciones
print(df[['Correo', 'Probabilidad renovación', 'Predicción renovación']].head())

# Guardar las predicciones en un archivo Excel
df[['Correo', 'Predicción renovación']].to_excel('/Users/juandiegoruizgonzalez/Desktop/Predicciones_Personalizadas.xlsx', index=False)




TypeError: can't multiply sequence by non-int of type 'float'

# 4. Exportar predicciones en xlsx

In [ ]:
# Verificar que el DataFrame tiene las predicciones actualizadas
print(df[['Correo', 'Probabilidad renovación']].head())

# Guardar las predicciones actualizadas en un archivo Excel
df[['Correo', 'Probabilidad renovación']].to_excel('/Users/juandiegoruizgonzalez/Desktop/Predicciones_Personalizadas.xlsx', index=False)


                       Correo  Probabilidad renovación
0               fod@siesa.com                        1
1        toyacampos@gmail.com                        1
2       wilmazafrat@gmail.com                        1
3  negretmosquera@hotmail.com                        1
4      hernanonatra@gmail.com                        1
